<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-4/3-reduce-feature-length-with-pca.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-4/3-reduce-feature-length-with-pca.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This code is part of [Chapter 4 - Building a Reverse Image Search Engine: Understanding Embeddings](https://learning.oreilly.com/library/view/practical-deep-learning/9781492034858/ch04.html).

Note: In order to run this notebook on Google Colab you need to [follow these instructions](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb#scrollTo=WzIRIt9d2huC) so that the local data such as the images are available in your Google Drive.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive

    drive.mount("/content/gdrive")

    IS_COLAB_ENV = True
except:
    IS_COLAB_ENV = False
IS_COLAB_ENV

Mounted at /content/gdrive


True

# Improving the speed of feature extraction and similarity search

We will experiment with PCA and figure out what is the optimum length of the features to use in our experiments.

## Reduce feature length with PCA

PCA (short for Principal Component Analysis) is a statistical procedure that questions if features representing the data are equally important. Are some of the features redundant enough that we can get similar results on say classification even after removing those features? PCA is considered one of the go-to techniques for dimensionality reduction. Note that it does not eliminate redundant features, instead it generates a new set of features that are a linear combination of the input features. These linear features are orthogonal to each other, which is why all the redundant features are absent. These features are known as principal components.


Performing PCA is pretty simple, using the scikit-learn library.

In [2]:
import numpy as np
import pickle
from tqdm import tqdm, tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors
import random
import glob

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

matplotlib.rcParams["savefig.dpi"] = 160
matplotlib.rcParams["figure.dpi"] = 160
%matplotlib notebook

You can run the following code on either the Caltech101 or the Caltech256 dataset. Run either one of the following two blocks of code to load the desired dataset.

In [3]:
model_architecture = "resnet"
model_features_path = f"data/features-caltech101-{model_architecture}.pickle"
generator_classes_path = "data/class_ids-caltech101.pickle"
filenames_path = "data/filenames-caltech101.pickle"

if IS_COLAB_ENV:
    generator_classes_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{generator_classes_path}"
    filenames_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filenames_path}"
    model_features_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{model_features_path}"

In [4]:
filenames = pickle.load(open(filenames_path, "rb"))
feature_list = pickle.load(open(model_features_path, "rb"))
class_ids = pickle.load(open(generator_classes_path, "rb"))

num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  8677
Number of features per image =  2048


In [5]:
def calculate_accuracy(feature_list, num_nearest_neighbors=5):
    neighbors = NearestNeighbors(
        n_neighbors=num_nearest_neighbors, algorithm="brute", metric="euclidean"
    ).fit(feature_list)
    distances, np_indices = neighbors.kneighbors(feature_list)

    start = time.time()
    neighbors_class_ids = class_ids[np_indices]
    class_equalities = np.equal(
        neighbors_class_ids[:, [0]], neighbors_class_ids[:, 1:num_nearest_neighbors]
    )
    accuracy = round(class_equalities.mean() * 100.0, 2)
    end = time.time()

    print(f"Accuracy is: {accuracy}")

    return accuracy, end - start

In [6]:
# Accuracy on original feature set
calculate_accuracy(feature_list)

Accuracy is: 88.46


(88.46, 0.0052797794342041016)

In [7]:
pca_dimensions = [1, 2, 3, 4, 5, 10, 20, 50, 75, 100, 150, 200]
pca_accuracy = []
pca_time = []

for dimensions in pca_dimensions:
    pca = PCA(n_components=dimensions)
    pca.fit(feature_list)
    feature_list_compressed = pca.transform(feature_list[:])
    # Calculate accuracy over the compressed features
    accuracy, t = calculate_accuracy(feature_list_compressed[:])
    pca_time.append(t)
    pca_accuracy.append(accuracy)
    print(
        "For PCA Dimensions = ",
        dimensions,
        ",\tAccuracy = ",
        accuracy,
        "%",
        ",\tTime = ",
        pca_time[-1],
    )

Accuracy is: 17.63
For PCA Dimensions =  1 ,	Accuracy =  17.63 % ,	Time =  0.0004937648773193359
Accuracy is: 26.49
For PCA Dimensions =  2 ,	Accuracy =  26.49 % ,	Time =  0.0008177757263183594
Accuracy is: 33.76
For PCA Dimensions =  3 ,	Accuracy =  33.76 % ,	Time =  0.0006639957427978516
Accuracy is: 40.53
For PCA Dimensions =  4 ,	Accuracy =  40.53 % ,	Time =  0.0007250308990478516
Accuracy is: 46.3
For PCA Dimensions =  5 ,	Accuracy =  46.3 % ,	Time =  0.0005846023559570312
Accuracy is: 66.41
For PCA Dimensions =  10 ,	Accuracy =  66.41 % ,	Time =  0.0006041526794433594
Accuracy is: 79.79
For PCA Dimensions =  20 ,	Accuracy =  79.79 % ,	Time =  0.0005383491516113281
Accuracy is: 86.8
For PCA Dimensions =  50 ,	Accuracy =  86.8 % ,	Time =  0.0004956722259521484
Accuracy is: 87.97
For PCA Dimensions =  75 ,	Accuracy =  87.97 % ,	Time =  0.0007348060607910156
Accuracy is: 88.6
For PCA Dimensions =  100 ,	Accuracy =  88.6 % ,	Time =  0.0005350112915039062
Accuracy is: 88.77
For PCA Dim

Let's plot the test accuracy on each PCA dimension.

In [8]:
f = plt.figure()

plt.plot(pca_dimensions, pca_accuracy, "o--", markersize=5)
plt.title("Test Accuracy v/s dimensions")
plt.xlabel("PCA Dimension Count")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

filename = "pca-dimensions_vs_acc"
if IS_COLAB_ENV:
    filename = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filename}"

f.savefig(f"{filename}.pdf", bbox_inches="tight")
f.savefig(f"{filename}.png", bbox_inches="tight")

<IPython.core.display.Javascript object>

As visible in the graph, there is little improvement in accuracy after increasing beyond feature length of 100 dimensions. With almost 20 times lesser dimensions (100) that the original (2048), this offers drastically higher speed and less time on almost any search algorithm, while getting similar (and sometimes slightly better) accuracy. Hence, 100 would be an ideal feature length for this dataset. This also means that the first 100 dimensions contain the most information about the dataset.


Let's plot the variance of each PCA dimension.

In [9]:
f = plt.figure()

plt.plot(range(1, 201), pca.explained_variance_ratio_, "o--", markersize=3)
plt.title("Variance v/s number of PCA dimensions")
plt.xlabel("PCA Dimension Count")
plt.ylabel("Variance")
plt.grid(True)
plt.show()

filename = "pca-variance_vs_dimensions"
if IS_COLAB_ENV:
    filename = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filename}"

f.savefig(f"{filename}.pdf", bbox_inches="tight")
f.savefig(f"{filename}.png", bbox_inches="tight")

<IPython.core.display.Javascript object>

The individual variance will tell us how important the newly added features are. For example, after the first 100 dimensions, the additional dimensions don’t add much variance (almost equal to 0) and can be neglected. Without even checking the accuracy it is safe to assume that the PCA with 100 dimensions will be a  robust model. Another way to look at this is to visualize how much of the original data is explained by the limited number of features by finding the cumulative variance.

### Cumulative Variance vs number of PCA dimensions.

In [10]:
f = plt.figure()

plt.plot(range(1, 201), pca.explained_variance_ratio_.cumsum(), "o--", markersize=3)
plt.title("Cumulative Variance v/s number of PCA dimensions")
plt.xlabel("PCA Dimension Count")
plt.ylabel("Cumulative Variance")
plt.grid(True)
plt.show()

filename = "pca-cumulativevariance_vs_dimensions"
if IS_COLAB_ENV:
    filename = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filename}"

f.savefig(f"{filename}.pdf", bbox_inches="tight")
f.savefig(f"{filename}.png", bbox_inches="tight")

<IPython.core.display.Javascript object>

As expected adding 100 dimensions (from 100 to 200) adds only .1 variance and starts to gradually plateau. For reference, using full 2048 features would result in a cumulative variance of 1.


Now let's compare different options available for similarity search

In [11]:
num_items = 100000
num_dimensions = 100

In [12]:
dataset = np.random.randn(num_items, num_dimensions)
dataset /= np.linalg.norm(dataset, axis=1).reshape(-1, 1)

In [13]:
randomIndex = random.randint(0, num_items)
query = dataset[randomIndex]

### Brute force

In [14]:
# Time the indexing for the brute force algorithm
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(dataset)

12.7 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
# Time the search for the brute force algorithm
neighbors = NearestNeighbors(n_neighbors=5, algorithm="brute", metric="euclidean").fit(
    dataset
)
%timeit neighbors.kneighbors([query])

22.9 ms ± 4.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### k-d Tree

In [16]:
# Time the indexing for the k-d tree algorithm
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(dataset)

2.25 s ± 758 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
# Time the search for the k-d tree algorithm
neighbors = NearestNeighbors(n_neighbors=5, algorithm="kd_tree").fit(dataset)
%timeit neighbors.kneighbors([query])

55.9 ms ± 8.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Ball Tree

In [18]:
# Time the indexing for the Ball Tree algorithm
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(dataset)

1.76 s ± 316 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
# Time the search for the Ball Tree algorithm
neighbors = NearestNeighbors(n_neighbors=5, algorithm="ball_tree").fit(dataset)
%timeit neighbors.kneighbors([query])

39.8 ms ± 6.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Now we have seen the time it takes to index and search using nearest neighbor algorithms on the full feature length. Now, We will use enhanced searching techniques like Faiss and Annoy to compress the features and reduce the time

### Annoy

Annoy (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings for searching nearest neighbors. Synonymous with speed, it was released by Spotify and is used in production to serve their music recommendations.

Make sure you have `annoy` installed. You can install it using pip, by executing the command below.

In [20]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552452 sha256=1f9a18e4bd2b361913e775c461b5b6439c9303a2e713e491af8a727e17be49e4
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [21]:
from annoy import AnnoyIndex

In [22]:
# Choose a random image to experiment
random_image_index = random.randint(0, num_items)
# Note: the results may change if the image is changed

First, we build a search index with two hyperparameters - the number of dimensions of the dataset, and the number of trees.

In [23]:
annoy_index = AnnoyIndex(
    num_dimensions, metric="angular"
)  # Length of item vector that will be indexed
for i in range(num_items):
    annoy_index.add_item(i, dataset[i])
annoy_index.build(40)  # 40 trees

True

Now let’s find out the time it takes to search the 5 nearest neighbors of one image.

In [24]:
# Time the search for one image for Annoy
%timeit annoy_index.get_nns_by_vector(query, 5, include_distances=True )

The slowest run took 4.24 times longer than the fastest. This could mean that an intermediate result is being cached.
83.8 µs ± 45.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Now THAT is blazing fast! To put this in perspective, for such a modestly sized dataset, this can serve almost 15000 requests on a single CPU core. Considering most CPUs have multiple cores, it should be able to handle 100K+ requests on a single system. The best part is that it lets you share the same index in memory between multiple processes. Hence, the biggest index can be equivalent to the size of your overall RAM, making it possible to serve multiple requests on a single system.

Other benefits include that it generates a modestly sized index. Moreover, it decouples creating indexes from loading them, so you can create an index on one machine, pass it around and then on your serving machine, load it in memory and serve it.

Let's time the indexing for different number of tree:

In [25]:
annoy_training_time = []
annoy_test_time = []
annoy_trees = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300]
random_image_index = random.randint(0, num_items)

for num_trees in annoy_trees:
    t = AnnoyIndex(
        num_dimensions, metric="angular"
    )  # Length of item vector that will be indexed
    for i in range(num_items):
        feature = dataset[i]
        t.add_item(i, feature)

    start_time = time.time()
    t.build(num_trees)
    end_time = time.time()
    annoy_training_time.append(end_time - start_time)

    start_time = time.time()
    indices = t.get_nns_by_vector(
        dataset[random_image_index], 5, include_distances=True
    )
    end_time = time.time()
    annoy_test_time.append(end_time - start_time)

    print(
        "For number of trees = ",
        num_trees,
        ",\tTime to train = ",
        annoy_training_time[-1],
        ",\tTime to test = ",
        annoy_test_time[-1],
    )

For number of trees =  1 ,	Time to train =  0.19180750846862793 ,	Time to test =  9.226799011230469e-05
For number of trees =  2 ,	Time to train =  0.34702348709106445 ,	Time to test =  9.417533874511719e-05
For number of trees =  3 ,	Time to train =  0.626061201095581 ,	Time to test =  9.751319885253906e-05
For number of trees =  4 ,	Time to train =  0.8793816566467285 ,	Time to test =  0.000152587890625
For number of trees =  5 ,	Time to train =  0.7698733806610107 ,	Time to test =  8.893013000488281e-05
For number of trees =  10 ,	Time to train =  1.362135887145996 ,	Time to test =  8.392333984375e-05
For number of trees =  20 ,	Time to train =  2.522669792175293 ,	Time to test =  0.0001761913299560547
For number of trees =  30 ,	Time to train =  4.48506498336792 ,	Time to test =  0.00014638900756835938
For number of trees =  40 ,	Time to train =  4.6653783321380615 ,	Time to test =  0.00018525123596191406
For number of trees =  50 ,	Time to train =  7.671112537384033 ,	Time to test

### Effect of number of trees vs Training time

In [26]:
plt.plot(annoy_trees, annoy_training_time, "or--")

f = plt.figure()

plt.plot(annoy_trees, annoy_training_time, "or--")
plt.title("Effect of number of trees vs Training time")
plt.xlabel("Number of trees")
plt.ylabel("Training Time")
plt.grid(True)
plt.show()

filename = "annoy-numtrees_vs_trainingtime"
if IS_COLAB_ENV:
    filename = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filename}"

f.savefig(f"{filename}.pdf", bbox_inches="tight")
f.savefig(f"{filename}.png", bbox_inches="tight")

<IPython.core.display.Javascript object>

### Effect of number of trees vs Test time

In [27]:
f = plt.figure()

plt.plot(annoy_trees, annoy_test_time, "or--")
plt.title("Effect of number of trees vs Test time")
plt.xlabel("Number of trees")
plt.ylabel("Test Time")
plt.grid(True)
plt.show()

filename = "annoy-numtrees_vs_testtime"
if IS_COLAB_ENV:
    filename = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filename}"

f.savefig(f"{filename}.pdf", bbox_inches="tight")
f.savefig(f"{filename}.png", bbox_inches="tight")

<IPython.core.display.Javascript object>

### NMS Lib

Make sure you have `nmslib` installed. You can install it using pip, by executing the command below.

In [28]:
!pip install nmslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578643 sha256=d15d5487c3ecf75e04327571993e482bfd82c84bfa33aef2018bc6de3a17ee8b
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib


In [29]:
import nmslib

In [30]:
index = nmslib.init(method="hnsw", space="cosinesimil")
index.addDataPointBatch(dataset)
index.createIndex({"post": 2}, print_progress=True)

In [31]:
# query for the nearest neighbors of the first datapoint
%timeit index.knnQuery(query, k=5)
ids, distances = index.knnQuery(query, k=5)

54.5 µs ± 15.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
# Get all nearest neighbors for all the datapoint
%timeit index.knnQueryBatch(dataset, k=5, num_threads=16)
neighbors = index.knnQueryBatch(dataset, k=5, num_threads=16)

10.9 s ± 1.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Falconn

Make sure you have `falconn` installed. You can install it using pip, by executing the command below.

In [33]:
!pip install falconn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for falconn: filename=FALCONN-1.3.1-cp310-cp310-linux_x86_64.whl size=14919689 sha256=796350496cc16f0e936ec4e3ba5336bbbe803cfe7fdc72ae6b33e7c9da9c382f
  Stored in directory: /root/.cache/pip/wheels/0b/4a/bc/68ac1e3cd3f263c47dfde8586fc3fdf704014ee3db0e5eb651
Successfully built falconn


In [34]:
import falconn

In [35]:
parameters = falconn.LSHConstructionParameters()
num_tables = 1
parameters.l = num_tables
parameters.dimension = num_dimensions
parameters.distance_function = falconn.DistanceFunction.EuclideanSquared
parameters.lsh_family = falconn.LSHFamily.CrossPolytope
parameters.num_rotations = 1
parameters.num_setup_threads = 1
parameters.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable
falconn.compute_number_of_hash_functions(16, parameters)

index = falconn.LSHIndex(parameters)
%time index.setup(dataset)

query_object = index.construct_query_object()
num_probes = 1
query_object.set_num_probes(num_probes)

%timeit query_object.find_k_nearest_neighbors(query, 5)

CPU times: user 132 ms, sys: 1 µs, total: 132 ms
Wall time: 135 ms
7.81 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
